# RAG langchain 說明
https://python.langchain.com/v0.1/docs/use_cases/question_answering/

offical sample code: [contextual_compression](https://python.langchain.com/docs/how_to/contextual_compression/)

In [5]:
from langchain_ollama import OllamaLLM
from langchain_ollama import OllamaEmbeddings

# 使用 ollama 的模型
# llm_model = OllamaLLM(
#     base_url='http://dandelion-ollama-1:11434', 
#     model="llama3.1:8b"
# )

# 使用 ollama 的 embedding 模型
embeddings_model = OllamaEmbeddings(
    base_url='http://dandelion-ollama-1:11434', 
    model="nomic-embed-text"
)


In [62]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 載入 PDF 文件
loader = PyPDFLoader('./data/red_bulid_dream.pdf')
# loader = PyPDFLoader('./data/operating_system_concepts.pdf')
docs = loader.load_and_split()

chunk_size = 256
chunk_overlap = 128

text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
documents = text_splitter.split_documents(docs)

ValueError: File path ./data/red_bulid_dream.pdf is not a valid file or url

In [7]:
def pretty_print_docs(docs):
    print("start print documents.\n")
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )

In [53]:
# import faiss
# from langchain_community.docstore.in_memory import InMemoryDocstore
# index = faiss.IndexFlatL2(len(embeddings_model.embed_query("hello world")))

# vector_store = FAISS(
#     embedding_function=embeddings_model,
#     index=index,
#     docstore=InMemoryDocstore(),
#     index_to_docstore_id={},
# )

# results = vector_store.similarity_search(
#     "小王子的朋友是誰？",
#     k=2,
#     filter={"source": "tweet"},
# )

In [54]:
# print(results)

[]


In [6]:
from langchain_community.document_loaders import TextLoader
# from langchain_community.document_loaders import PyPDFLoader
# from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
# from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter

documents = TextLoader("./../data/red.txt").load()
text_splitter = CharacterTextSplitter(chunk_size=150, chunk_overlap=0)
texts = text_splitter.split_documents(documents)
retriever = FAISS.from_documents(texts, embeddings_model).as_retriever()

docs = retriever.invoke("賈寶玉與林黛玉的關係？")
pretty_print_docs(docs)

Created a chunk of size 3749, which is longer than the specified 150
Created a chunk of size 8266, which is longer than the specified 150
Created a chunk of size 6302, which is longer than the specified 150
Created a chunk of size 14700, which is longer than the specified 150
Created a chunk of size 7885, which is longer than the specified 150
Created a chunk of size 7459, which is longer than the specified 150
Created a chunk of size 7476, which is longer than the specified 150
Created a chunk of size 6618, which is longer than the specified 150
Created a chunk of size 5644, which is longer than the specified 150
Created a chunk of size 5102, which is longer than the specified 150
Created a chunk of size 5901, which is longer than the specified 150
Created a chunk of size 4272, which is longer than the specified 150
Created a chunk of size 5180, which is longer than the specified 150
Created a chunk of size 5617, which is longer than the specified 150
Created a chunk of size 5053, whi

NameError: name 'pretty_print_docs' is not defined

## Adding contextual compression with an LLMChainExtractor

Now let's wrap our base retriever with a ContextualCompressionRetriever. We'll add an LLMChainExtractor, which will iterate over the initially returned documents and extract from each only the content that is relevant to the query.

[ContextualCompressionRetriever](https://python.langchain.com/api_reference/langchain/retrievers/langchain.retrievers.contextual_compression.ContextualCompressionRetriever.html)

[LLMChainExtractor](!https://python.langchain.com/api_reference/langchain/retrievers/langchain.retrievers.document_compressors.chain_extract.LLMChainExtractor.html)

In [71]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

# llm = OpenAI(temperature=0)
llm = OllamaLLM(
    base_url='http://dandelion-ollama-1:11434', 
    model="llama3.1:8b",
    temperature=0
)

compressor = LLMChainExtractor.from_llm(llm)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

compressed_docs = compression_retriever.invoke(
    "賈寶玉與林黛玉的關係？"
)
pretty_print_docs(compressed_docs)

start print documents.

Document 1:

> 賈寶玉與林黛玉的關係？

> 傲世也因同氣味，春風桃李未淹留。
----------------------------------------------------------------------------------------------------
Document 2:

Here are the extracted relevant parts:

**Part 1**

* Lin Daiyu (林黛玉) is upset with Baoyu (宝玉) for not seeing her yesterday and is now avoiding him.
* Baoyu tries to guess what's wrong but can't figure it out.

**Part 2**

* Baoyu meets up with his sister, Xichun (西春), who has been saving money to buy some nice things.
* Xichun asks Baoyu to bring her back some nice items when he goes out, such as a pair of shoes made by her.
* Baoyu tells her about the time he wore a pair of shoes made by her and was scolded by their father for wasting money on frivolous things.

**Part 3**

* Xichun is upset with Baoyu for not understanding her feelings and is now avoiding him too.
* Baoyu tries to make amends by bringing her some flowers from the garden, but she's still upset.

**Part 4**

* Baoyu decides to go to the

In [7]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

# llm = OpenAI(temperature=0)
llm = OllamaLLM(
    base_url='http://dandelion-ollama-1:11434', 
    model="llama3.1:8b",
    temperature=0
)

compressor = LLMChainExtractor.from_llm(llm)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

compressed_docs = compression_retriever.invoke(
    "誰可以佔有星星？"
)
pretty_print_docs(compressed_docs)


Document 1:

>誰可以佔有星星？
 
“什么也不做。它們都是屬于我的。” 
 
“星星是屬于你的？” 
 
“是的。” 
 
“你擁有這許多星星有什么用？” 
 
“富了就可以去買別的星星，如果有人發現了別的星星的話。”
----------------------------------------------------------------------------------------------------
Document 2:

實業家不高興地頂了小王子一句。
“那么，你說星星是誰的呀？”
“我不知道，不屬于任何人。”
“你既然知道它們不屬于任何人，那么，它們就是我的，因為是我第一個想到了這件事情的。”
----------------------------------------------------------------------------------------------------
Document 3:

> 中一顆星星上笑著，那么對你來說，就好象所有的星星都在笑，那么你將看到的 
星星就是會笑的星星！” 
 
    這時，他又笑了。 
 
    “那么，在你得到了安慰之后（人們總是會自我安慰的） 你就會因為認識了 
我而感到高興。你將永遠是我的朋友。你就會想要同我一起笑。有時，你會為了 
快樂而不知不覺地打開窗戶。你的朋友們會奇怪地看著你笑著仰望天空。那時， 
你就可以對他們說：‘是的，星星總是引我歡笑！’他們會以為你發瘋了。我的 
惡作劇將使你難堪…”
----------------------------------------------------------------------------------------------------
Document 4:

> 小王子對一些無關緊要的話看得太認真，結果使自己很苦惱。
> “我那時什么也不懂！我應該根據她的行為，而不是根據她的話來判斷她。”
> “花是多么自相矛盾！我當時太年青， 還不懂得愛她。 ”


## More built-in compressors: filters
The LLMChainFilter is slightly simpler but more robust compressor that uses an LLM chain to decide which of the initially retrieved documents to filter out and which ones to return, without manipulating the document contents.

[LLMChainFilter](https://python.langchain.com/api_reference/langchain/retrievers/langchain.retrievers.document_compressors.chain_filter.LLMChainFilter.html)

In [67]:
from langchain.retrievers.document_compressors import LLMChainFilter

_filter = LLMChainFilter.from_llm(llm)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=_filter, base_retriever=retriever
)

compressed_docs = compression_retriever.invoke(
    "賈政是誰？"
)
pretty_print_docs(compressed_docs)

start print documents.




In [10]:
from langchain.retrievers.document_compressors import LLMChainFilter

_filter = LLMChainFilter.from_llm(llm)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=_filter, base_retriever=retriever
)

compressed_docs = compression_retriever.invoke(
    "誰可以佔有星星？"
)
pretty_print_docs(compressed_docs)

ValueError: Ambiguous response. Both YES and NO in received: YES

The context is a conversation between a small prince and an industrialist about who can own stars. The industrialist claims that since no one has thought of owning stars before, he can own them by being the first to think of it. The prince questions this idea, pointing out that you can't actually take possession of stars like you would with physical objects.

This conversation is directly relevant to the question "誰可以佔有星星?" (Who can own stars?)..

In [11]:
from langchain.retrievers.document_compressors import LLMChainFilter

_filter = LLMChainFilter.from_llm(llm)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=_filter, base_retriever=retriever
)

compressed_docs = compression_retriever.invoke(
    "玫瑰跟狐狸誰跟小王子關係比較好？"
)
pretty_print_docs(compressed_docs)

start print documents.

Document 1:

36 
    “來和我一起玩吧，”小王子建議道，“我很苦惱…” 
 
    “我不能和你一起玩，”狐狸說，“我還沒有被馴服呢。” 
 
    “啊！真對不起。”小王子說。 
 
    思索了一會兒，他又說道： 
 
    “什么叫‘馴服’呀？” 
 
    “你不是此地人。”狐狸說，“你來尋找什么？” 
 
    “我來找人。”小王子說，“什么叫‘馴服’呢？” 
 
    “人，”狐狸說，“他們有槍，他們還打獵，這真礙事！他們唯一的可取之 
處就是他們也養雞，你是來尋找雞的嗎？” 
 
    “不，”小王子說，“我是來找朋友的。什么叫‘馴服’呢？” 
 
    “這是已經早就被人遺忘了的事情，”狐狸說，“它的意思就是‘建立聯
系’。” 
 
    “建立聯系？” 
 
    “一點不錯，”狐狸說。“對我來說，你還只是一個小男孩，就像其他千萬 
個小男孩一樣。我不需要你。你也同樣用不著我。對你來說，我也不過是一只狐 
狸，和其他千萬只狐狸一樣。但是，如果你馴服了我，我們就互相不可缺少了。 
對我來說，你就是世界上唯一的了﹔我對你來說，也是世界上唯一的了。” 
 
    “我有點明白了。”小王子說，“有一朵花…，我想，她把我馴服了…” 
 
    “這是可能的。”狐狸說，“世界上什么樣的事都可能看到…” 
 
    “啊，這不是在地球上的事。”小王子說。 
 
    狐狸感到十分蹊蹺。 
 
    “在另一個星球上？”


### LLMListwiseRerank  
LLMListwiseRerank uses [zero-shot listwise document reranking](https://arxiv.org/pdf/2305.02156) and functions similarly to LLMChainFilter as a robust but more expensive option. It is recommended to use a more powerful LLM.

Note that LLMListwiseRerank requires a model with the with_structured_output method implemented.

[LLMListwiseRerank](https://python.langchain.com/api_reference/langchain/retrievers/langchain.retrievers.document_compressors.listwise_rerank.LLMListwiseRerank.html)

In [24]:
# from langchain_community.llms.ollama import OllamaFunctions
from langchain.retrievers.document_compressors import LLMListwiseRerank
# from langchain_community.chat_models import OllamaFunctions
# from langchain_experimental.llms import OllamaFunctions
from langchain_core.pydantic_v1 import BaseModel
from langchain_ollama import ChatOllama

# llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 使用 OllamaFunctions 實例化 LLM
llm_for_rerank = ChatOllama(
    base_url='http://dandelion-ollama-1:11434',
    model="llama3.1:8b",
    format="json",
    temperature=0
)

_filter = LLMListwiseRerank.from_llm(llm_for_rerank, top_n=10)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=_filter, base_retriever=retriever
)
# with_structured_output
compressed_docs = compression_retriever.invoke(
    "What is os？"
)
pretty_print_docs(compressed_docs)

/opt/conda/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3553: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


start print documents.

Document 1:

﻿紅樓夢

Exported from Wikisource on 2020年12月17日

本書是前八十回基於脂本，後四十回基於程高本的通行版一百二十回《紅樓夢》
如您想讀附帶脂批的古本八十回《紅樓夢》，可參見「脂硯齋重評石頭記」。
----------------------------------------------------------------------------------------------------
Document 2:

欲訊秋情眾莫知，喃喃負手叩東篱。
　　　　孤標傲世偕誰隱，一樣花開為底遲？
　　　　圃露庭霜何寂寞，鴻歸蛩病可相思？
　　　　休言舉世無談者，解語何妨片語時。


　　　　　　　　簪菊
　　　　　　　　〈蕉下客〉
----------------------------------------------------------------------------------------------------
Document 3:

詩餘戲筆不知狂，豈是丹青費較量。
　　　　聚葉潑成千點墨，攢花染出幾痕霜。
　　　　淡濃神會風前影，跳脫秋生腕底香。
　　　　莫認東篱閒采掇，粘屏聊以慰重陽。


　　　　　　　　問菊
　　　　　　　　〈瀟湘妃子〉


In [45]:
# from langchain_community.llms.ollama import OllamaFunctions
from langchain.retrievers.document_compressors import LLMListwiseRerank
# from langchain_community.chat_models import OllamaFunctions
# from langchain_experimental.llms import OllamaFunctions
from langchain_core.pydantic_v1 import BaseModel
from langchain_ollama import ChatOllama

# llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 使用 OllamaFunctions 實例化 LLM
llm_for_rerank = ChatOllama(
    base_url='http://dandelion-ollama-1:11434',
    model="llama3.1:8b",
    format="json",
    temperature=0
)

_filter = LLMListwiseRerank.from_llm(llm_for_rerank, top_n=10)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=_filter, base_retriever=retriever
)
# with_structured_output
compressed_docs = compression_retriever.invoke(
    "小王子的朋友是誰？"
)
pretty_print_docs(compressed_docs)

start print documents.

Document 1:

36 
    “來和我一起玩吧，”小王子建議道，“我很苦惱…” 
 
    “我不能和你一起玩，”狐狸說，“我還沒有被馴服呢。” 
 
    “啊！真對不起。”小王子說。 
 
    思索了一會兒，他又說道： 
 
    “什么叫‘馴服’呀？” 
 
    “你不是此地人。”狐狸說，“你來尋找什么？” 
 
    “我來找人。”小王子說，“什么叫‘馴服’呢？” 
 
    “人，”狐狸說，“他們有槍，他們還打獵，這真礙事！他們唯一的可取之 
處就是他們也養雞，你是來尋找雞的嗎？” 
 
    “不，”小王子說，“我是來找朋友的。什么叫‘馴服’呢？” 
 
    “這是已經早就被人遺忘了的事情，”狐狸說，“它的意思就是‘建立聯
系’。” 
 
    “建立聯系？” 
 
    “一點不錯，”狐狸說。“對我來說，你還只是一個小男孩，就像其他千萬 
個小男孩一樣。我不需要你。你也同樣用不著我。對你來說，我也不過是一只狐 
狸，和其他千萬只狐狸一樣。但是，如果你馴服了我，我們就互相不可缺少了。 
對我來說，你就是世界上唯一的了﹔我對你來說，也是世界上唯一的了。” 
 
    “我有點明白了。”小王子說，“有一朵花…，我想，她把我馴服了…” 
 
    “這是可能的。”狐狸說，“世界上什么樣的事都可能看到…” 
 
    “啊，這不是在地球上的事。”小王子說。 
 
    狐狸感到十分蹊蹺。 
 
    “在另一個星球上？”
----------------------------------------------------------------------------------------------------
Document 2:

21 
 
    “讓我高興吧，請你還是來欽佩我吧！” 
 
    小王子輕輕地聳了聳肩膀，說道：“我欽佩你，可是，這有什么能使你這樣 
感興趣的？” 
 
    于是小王子就走開了。 
 
    小王子在路上自言自語地說了一句：“這些大人，肯定是十分古怪的。” 
 
 
                                XII 
 
    小王子所訪問的下一個星球上住著一個酒鬼。訪問時間非常短，可是

### EmbeddingsFilter

Making an extra LLM call over each retrieved document is expensive and slow. The EmbeddingsFilter provides a cheaper and faster option by embedding the documents and query and only returning those documents which have sufficiently similar embeddings to the query.

[EmbeddingsFilter](https://python.langchain.com/api_reference/langchain/retrievers/langchain.retrievers.document_compressors.embeddings_filter.EmbeddingsFilter.html)

In [14]:
from langchain.retrievers.document_compressors import EmbeddingsFilter
from langchain.retrievers import ContextualCompressionRetriever
from langchain_ollama import OllamaEmbeddings

embeddings = OllamaEmbeddings(
    base_url='http://dandelion-ollama-1:11434', 
    model="nomic-embed-text"
)
embeddings_filter = EmbeddingsFilter(embeddings=embeddings, similarity_threshold=0.76)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=embeddings_filter, base_retriever=retriever
)

compressed_docs = compression_retriever.invoke(
    # "賈政是誰？"
    "紅樓夢？"
)
pretty_print_docs(compressed_docs)

start print documents.




### Stringing compressors and document transformers together
Using the DocumentCompressorPipeline we can also easily combine multiple compressors in sequence. Along with compressors we can add BaseDocumentTransformers to our pipeline, which don't perform any contextual compression but simply perform some transformation on a set of documents. For example TextSplitters can be used as document transformers to split documents into smaller pieces, and the EmbeddingsRedundantFilter can be used to filter out redundant documents based on embedding similarity between documents.

Below we create a compressor pipeline by first splitting our docs into smaller chunks, then removing redundant documents, and then filtering based on relevance to the query.

In [20]:
from langchain.retrievers.document_compressors import DocumentCompressorPipeline
from langchain_community.document_transformers import EmbeddingsRedundantFilter
from langchain_text_splitters import CharacterTextSplitter

splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=0, separator=". ")
redundant_filter = EmbeddingsRedundantFilter(embeddings=embeddings)
relevant_filter = EmbeddingsFilter(embeddings=embeddings, similarity_threshold=0)
pipeline_compressor = DocumentCompressorPipeline(
    transformers=[splitter, redundant_filter, relevant_filter]
)

In [21]:
compression_retriever = ContextualCompressionRetriever(
    base_compressor=pipeline_compressor, base_retriever=retriever
)

compressed_docs = compression_retriever.invoke(
    "寶玉"
)
pretty_print_docs(compressed_docs)

start print documents.

Document 1:

秋光疊疊复重重，潛度偷移三徑中。
　　　　窗隔疏燈描遠近，篱篩破月鎖玲瓏。
　　　　寒芳留照魂應駐，霜印傳神夢也空。
　　　　珍重暗香休踏碎，憑誰醉眼認朦朧。


　　　　　　　　菊夢
　　　　　　　　〈瀟湘妃子〉
----------------------------------------------------------------------------------------------------
Document 2:

別圃移來貴比金，一叢淺淡一叢深。
　　　　蕭疏篱畔科頭坐，清冷香中抱膝吟。
　　　　數去更無君傲世，看來惟有我知音。
　　　　秋光荏苒休辜負，相對原宜惜寸陰。


　　　　　　　　供菊
　　　　　　　　〈枕霞舊友〉
----------------------------------------------------------------------------------------------------
Document 3:

彈琴酌酒喜堪儔，几案婷婷點綴幽。
　　　　隔座香分三徑露，拋書人對一枝秋。
　　　　霜清紙帳來新夢，圃冷斜陽憶舊游。
　　　　傲世也因同氣味，春風桃李未淹留。


　　　　　　　　詠菊
　　　　　　　　〈瀟湘妃子〉
----------------------------------------------------------------------------------------------------
Document 4:

詩餘戲筆不知狂，豈是丹青費較量。
　　　　聚葉潑成千點墨，攢花染出幾痕霜。
　　　　淡濃神會風前影，跳脫秋生腕底香。
　　　　莫認東篱閒采掇，粘屏聊以慰重陽。


　　　　　　　　問菊
　　　　　　　　〈瀟湘妃子〉


In [28]:
from langchain.retrievers.document_compressors import LLMListwiseRerank
from langchain.retrievers import ContextualCompressionRetriever
from langchain_ollama import ChatOllama

llm_for_rerank = ChatOllama(
    base_url='http://dandelion-ollama-1:11434',
    model="llama3.1:8b",
    format="json",
    temperature=0
)

# ¨Ï¥Î LLMListwiseRerank §@¬°À£ÁY¾¹
# ¥¦·|ªð¦^¤@­Ó¥]§t¤À¼Æªº±Æ§Ç¤å¥ó¦Cªí
_filter_with_scores = LLMListwiseRerank.from_llm(llm_for_rerank, top_n=5) # top_n ¨M©wªð¦^¦h¤Ö­Ó¤å¥ó

compression_retriever_with_scores = ContextualCompressionRetriever(
    base_compressor=_filter_with_scores, base_retriever=retriever
)

# °õ¦æ®É¡ALLMListwiseRerank ·|¦b¤º³¡³B²zµû¤À©M±Æ§Ç
# ³Ì²×ªð¦^ªº¤å¥ó³q±`·|«ö·Ó¤À¼Æ°ª§C±Æ§Ç¡A¥B¥i¯à¦b Document ªº metadata ¤¤¥]§t¤À¼Æ«H®§
compressed_docs_with_scores = compression_retriever_with_scores.invoke(
    "誰是寶玉？"
)

for doc in compressed_docs_with_scores:
    print(f"Document: {doc.page_content[:100]}...")
    print(f"Score: {doc.metadata.get('relevance_score', 'N/A')}")
    print("-" * 20)

Document: 篱畔秋酣一覺清，和雲伴月不分明。
　　　　登仙非慕庄生蝶，憶舊還尋陶令盟。
　　　　睡去依依隨雁斷，驚回故故惱蛩鳴。
　　　　醒時幽怨同誰訴，衰草寒煙無限情。


　　　　　　　　殘菊
　　　　　　　...
Score: N/A
--------------------
Document: 攜鋤秋圃自移來，篱畔庭前故故栽。
　　　　昨夜不期經雨活，今朝猶喜帶霜開。
　　　　冷吟秋色詩千首，醉酹寒香酒一杯。
　　　　泉溉泥封勤護惜，好知井徑絕塵埃。


　　　　　　　　對菊
　　　　　　　...
Score: N/A
--------------------
Document: 欲訊秋情眾莫知，喃喃負手叩東篱。
　　　　孤標傲世偕誰隱，一樣花開為底遲？
　　　　圃露庭霜何寂寞，鴻歸蛩病可相思？
　　　　休言舉世無談者，解語何妨片語時。


　　　　　　　　簪菊
　　　　　　　...
Score: N/A
--------------------
Document: 彈琴酌酒喜堪儔，几案婷婷點綴幽。
　　　　隔座香分三徑露，拋書人對一枝秋。
　　　　霜清紙帳來新夢，圃冷斜陽憶舊游。
　　　　傲世也因同氣味，春風桃李未淹留。


　　　　　　　　詠菊
　　　　　　　...
Score: N/A
--------------------


In [ ]:
# from langchain_community.llms.ollama import OllamaFunctions
from langchain.retrievers.document_compressors import LLMListwiseRerank
# from langchain_community.chat_models import OllamaFunctions
# from langchain_experimental.llms import OllamaFunctions
from langchain_core.pydantic_v1 import BaseModel
from langchain_ollama import ChatOllama

# llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 使用 OllamaFunctions 實例化 LLM
llm_for_rerank = ChatOllama(
    base_url='http://dandelion-ollama-1:11434',
    model="llama3.1:8b",
    format="json",
    temperature=0
)

_filter = LLMListwiseRerank.from_llm(llm_for_rerank, top_n=10)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=_filter, base_retriever=retriever
)
# with_structured_output
compressed_docs = compression_retriever.invoke(
    "What is os？"
)
pretty_print_docs(compressed_docs)